# ARBOL DE DECISIÓN CON PODA

Librerias

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, auc, RocCurveDisplay, PrecisionRecallDisplay

Obtención de los datos

In [ ]:
# Cargar el archivo CSV
file_path = 'supervisado_final.csv'
df = pd.read_csv(file_path, sep=';', decimal='.')

# Eliminar la columna ID Usuario y separar las características del objetivo (Conclusion)
X = df.drop(columns=["ID Usuario", "Conclusion"])
y = df["Conclusion"].map({0: 0, 0.5: 1, 1: 2})

Divir los datos en train (70%), test (20%) y validación (10%)

In [ ]:
# Dividir en conjunto de entrenamiento (70%), validación (10%) y prueba (20%)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.125, random_state=42)  # 0.125 x 0.8 = 0.1

Definir el árbol de decisión

In [ ]:
# Definir el Árbol de Decisión con una búsqueda extensa de hiperparámetros
clf_tree = DecisionTreeClassifier(random_state=42)

Campo de busqueda de híperparametros, se ha descomentado la mejor opción

In [ ]:
# Parámetros para la búsqueda en malla
# param_grid_tree = {
#     'criterion': ['gini', 'entropy'],                  # Comparar criterios de evaluación
#     'max_depth': [None, 5, 10, 15, 20, 25, 30, 35],        # Aumentar el rango de la profundidad máxima
#     'min_samples_split': [2, 5, 10, 15, 20, 25, 30],       # Más valores para las muestras mínimas al dividir
#     'min_samples_leaf': [1, 2, 5, 10, 15, 20],         # Variar las muestras mínimas en una hoja
#     'ccp_alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.2]  # Extender el rango de poda de coste-complejidad
# }

param_grid_tree = {
    'criterion': ['entropy'],                  # Comparar criterios de evaluación
    'max_depth': [None],        # Aumentar el rango de la profundidad máxima
    'min_samples_split': [2],       # Más valores para las muestras mínimas al dividir
    'min_samples_leaf': [10],         # Variar las muestras mínimas en una hoja
    'ccp_alpha': [0.0]  # Extender el rango de poda de coste-complejidad
}

Busqueda en malla con el conjunto de validación con GridSearch

In [ ]:
# Búsqueda en malla con el conjunto de validación
grid_search_tree = GridSearchCV(clf_tree, param_grid_tree, cv=5, scoring='accuracy')
grid_search_tree.fit(X_train, y_train)

Seleccionar la mejor opción y evaluar el modelo

In [ ]:
# Seleccionar el mejor Árbol de Decisión
tree_best = grid_search_tree.best_estimator_

# Evaluar el mejor modelo en el conjunto de validación
y_val_pred = tree_best.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_classification = classification_report(y_val, y_val_pred)

# Mostrar resultados del mejor Árbol de Decisión en el conjunto de validación
print(f"Mejor modelo Árbol de Decisión: {tree_best}")
# Imprimir cada parámetro por separado
print(f"Mejor criterio: {grid_search_tree.best_params_['criterion']}")
print(f"Mejor max_depth: {grid_search_tree.best_params_['max_depth']}")
print(f"Mejor min_samples_split: {grid_search_tree.best_params_['min_samples_split']}")
print(f"Mejor min_samples_leaf: {grid_search_tree.best_params_['min_samples_leaf']}")
print(f"Mejor ccp_alpha: {grid_search_tree.best_params_['ccp_alpha']}")


# Evaluar el modelo en el conjunto de prueba
y_test_pred = tree_best.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_classification = classification_report(y_test, y_test_pred)

# Mostrar resultados del mejor Árbol de Decisión en el conjunto de prueba
print(f"Precisión en Prueba Arbol: {test_accuracy}")
print("Reporte de clasificación en Prueba:")
print(test_classification)

Visualizar el árbol de decisión seleccionado

In [ ]:
# Visualizar el mejor Árbol de Decisión
plt.figure(figsize=(20, 10))
plot_tree(tree_best, feature_names=X.columns.tolist(), class_names=["0", "0.5", "1"], filled=True, rounded=True)
plt.title("Mejor Árbol de Decisión")
plt.show()

Visualización de la importancia de las características en consola y en diagrama de barras

In [ ]:
# Importancia de las características
importances = tree_best.feature_importances_
sorted_indices = np.argsort(importances)[::-1]  # Ordenar de mayor a menor importancia

# Imprimir las características principales ordenadas por importancia
print("\nCaracterísticas principales ordenadas por importancia:")
for index in sorted_indices:
    print(f"{X.columns[index]}: {importances[index]:.4f}")

# Gráfico de la importancia de las características
plt.figure(figsize=(12, 6))
plt.barh(X.columns[sorted_indices], importances[sorted_indices])
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.title('Importancia de las características - Árbol de Decisión')
plt.show()

Matriz de confusión para evaluar el rendimiento con los datos de test

In [ ]:
# Mostrar la Matriz de Confusión para evaluar el rendimiento del modelo en el conjunto de prueba
conf_matrix = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=["0 (Bien)", "0.5 (Revisar)", "1 (Mal)"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusión - Árbol de Decisión")
plt.show()

Grafica de la curva de complejidad del árbol de decisión (No se analiza en la memoria)

In [ ]:
# Generar árboles con diferentes valores de ccp_alpha
path = tree_best.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Entrenar un modelo para cada valor de ccp_alpha
trees = []
train_scores = []
val_scores = []

for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    trees.append(clf)
    train_scores.append(clf.score(X_train, y_train))
    val_scores.append(clf.score(X_val, y_val))

# Graficar la curva de complejidad
plt.figure(figsize=(10, 6))
plt.plot(ccp_alphas, train_scores, marker='o', label='Accuracy en Entrenamiento', drawstyle="steps-post")
plt.plot(ccp_alphas, val_scores, marker='o', label='Accuracy en Validación', drawstyle="steps-post")
plt.xlabel('Valor de ccp_alpha (Poda de Complejidad)')
plt.ylabel('Accuracy')
plt.title('Curva de Complejidad del Árbol de Decisión')
plt.legend()
plt.grid()
plt.show()

Curva precision-recall

In [ ]:
# Obtener las probabilidades de las predicciones en el conjunto de prueba
y_test_proba = tree_best.predict_proba(X_test)

# Obtener las etiquetas binarizadas para cada clase
n_classes = len(np.unique(y))
y_test_bin = np.zeros((y_test.size, n_classes))
for i in range(n_classes):
    y_test_bin[:, i] = (y_test == i).astype(int)

# Graficar todas las curvas Precision-Recall en el mismo gráfico
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_test_proba[:, i])
    plt.plot(recall, precision, lw=2, label=f'Clase {i}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall para cada clase')
plt.legend(loc='best')
plt.grid()
plt.show()

Curva ROC y AUC

In [ ]:
# Graficar todas las curvas ROC en el mismo gráfico
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_test_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'Clase {i} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC para cada clase')
plt.legend(loc='best')
plt.grid()
plt.show()

## Aplicamos SMOTE

Se aplica SMOTE para balancear las clases, para más información leer la memoria.

In [ ]:
from imblearn.over_sampling import SMOTE

Aplicar SMOTE

In [ ]:
# Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Revisar la distribución de clases después de aplicar SMOTE
print("Distribución de clases después de aplicar SMOTE:")
print(y_train_smote.value_counts())

Volver a ajustar el modelo a los nuevos datos sinteticos

In [ ]:
# Volver a ajustar el modelo de Árbol de Decisión con los datos balanceados
grid_search_tree.fit(X_train_smote, y_train_smote)

Selección del mejor árbol de decisión (solo se usa la misma configuración que el arbol sin SMOTE) y evaluación del mismo

In [ ]:
# Seleccionar el mejor Árbol de Decisión tras el ajuste con SMOTE
tree_with_smote = grid_search_tree.best_estimator_
y_pred_tree_smote = tree_with_smote.predict(X_test)

# Evaluar precisión y mostrar el reporte de clasificación con los datos ajustados
accuracy_tree_smote = accuracy_score(y_test, y_pred_tree_smote)
classification_tree_smote = classification_report(y_test, y_pred_tree_smote)

print("\nMejor modelo Árbol de Decisión tras aplicar SMOTE: ", tree_with_smote)
print(f"Precisión Árbol de Decisión tras aplicar SMOTE: {accuracy_tree_smote:.4f}")
print("Reporte de clasificación tras aplicar SMOTE:")
print(classification_tree_smote)

Visualizar el árbol con SMOTE

In [ ]:
# Visualizar el mejor Árbol de Decisión con SMOTE
plt.figure(figsize=(20, 10))
plot_tree(tree_with_smote, feature_names=X.columns.tolist(), class_names=["0", "0.5", "1"], filled=True, rounded=True)
plt.title("Mejor Árbol de Decisión con Poda (Con SMOTE)")
plt.show()

Matriz de confusión del arbol de decisión entrenado con SMOTE

In [ ]:
# Mostrar la Matriz de Confusión para evaluar el rendimiento del modelo con SMOTE
conf_matrix = confusion_matrix(y_test, y_pred_tree_smote)
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=["0 (Bien)", "0.5 (Revisar)", "1 (Mal)"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusión - Árbol de Decisión con SMOTE")
plt.show()

Visualización de las importancias de las caracteristicas por consola y en diagrama de barras para los datos sinteticos

In [ ]:
# Importancia de las características (con SMOTE)
importances = tree_with_smote.feature_importances_
sorted_indices = np.argsort(importances)[::-1]  # Ordenar de mayor a menor importancia

# Imprimir las características principales ordenadas por importancia
print("\nCaracterísticas principales ordenadas por importancia (Con SMOTE):")
for index in sorted_indices:
    print(f"{X.columns[index]}: {importances[index]:.4f}")

# Gráfico de la importancia de las características (con SMOTE)
plt.figure(figsize=(12, 6))
plt.barh(X.columns[sorted_indices], importances[sorted_indices])
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.title('Importancia de las características - Árbol de Decisión con SMOTE')
plt.show()

Curva ROC y AUC para el arbol entrenado con SMOTE

In [ ]:
# Evaluación con curvas ROC y AUC (con SMOTE)
y_prob_smote = tree_with_smote.predict_proba(X_test)
fpr = {}
tpr = {}
roc_auc = {}

# Calculamos ROC y AUC para cada clase
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test == i, y_prob_smote[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Gráfico de la curva ROC (con SMOTE)
plt.figure(figsize=(10, 8))
for i in range(3):
    plt.plot(fpr[i], tpr[i], label=f'Clase {i} (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curvas ROC para cada clase - Árbol de Decisión con SMOTE')
plt.legend()
plt.show()

Visualizar curva de complejidad del árbol de decisión con SMOTE

In [ ]:
# Generar árboles con diferentes valores de ccp_alpha con SMOTE
path_smote = tree_with_smote.cost_complexity_pruning_path(X_train_smote, y_train_smote)
ccp_alphas_smote, impurities_smote = path_smote.ccp_alphas, path_smote.impurities

# Entrenar un modelo para cada valor de ccp_alpha (Con SMOTE)
trees_smote = []
train_scores_smote = []
test_scores_smote = []

for ccp_alpha in ccp_alphas_smote:
    clf = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha)
    clf.fit(X_train_smote, y_train_smote)
    trees_smote.append(clf)
    train_scores_smote.append(clf.score(X_train_smote, y_train_smote))
    test_scores_smote.append(clf.score(X_test, y_test))

# Graficar la curva de complejidad con SMOTE
plt.figure(figsize=(10, 6))
plt.plot(ccp_alphas_smote, train_scores_smote, marker='o', label='Accuracy en Entrenamiento', drawstyle="steps-post")
plt.plot(ccp_alphas_smote, test_scores_smote, marker='o', label='Accuracy en Prueba', drawstyle="steps-post")
plt.xlabel('Valor de ccp_alpha (Poda de Complejidad)')
plt.ylabel('Accuracy')
plt.title('Curva de Complejidad del Árbol de Decisión con SMOTE')
plt.legend()
plt.grid()
plt.show()